In [104]:
import requests
import urllib
from datetime import datetime
import pandas as pd
import time
import asyncio

BASE_URL = 'https://api.prop-odds.com'
API_KEY = 'jQWNvuSMdB1SiXsWxDgmRzzHUMDQyQXBLZ95WmFI'

#### first four methods are adapted from prop-odds @ https://github.com/M4THYOU/prop_odds_python_example/blob/main/main.py
def get_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

    # print('Request failed with status:', response.status_code)
    return {}


def get_nfl_games():
    now = datetime.now()
    query_params = {
        'date': now.strftime('%Y-%m-%d'),
        'tz': 'America/New_York',
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/games/nfl?' + params
    return get_request(url)

def get_nba_games():
    now = datetime.now()
    query_params = {
        'date': now.strftime('%Y-%m-%d'),
        'tz': 'America/New_York',
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/games/nba?' + params
    return get_request(url)


def get_most_recent_odds(game_id, market):
    query_params = {
        'api_key': API_KEY,
        'active_only': False,
        'end_date_time': "2024-01-08T00:00:00Z"
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/odds/' + game_id + '/' + market + '?' + params
    return get_request(url)


def get_fantasy_lines(game_id, market):
    query_params = {
        'api_key': API_KEY,
        'active_only': False,
        'end_date_time': "2024-01-08T00:00:00Z"
    }

    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/fantasy_lines/' + game_id + '/' + market + '?' + params
    return get_request(url)


In [2]:
#### processing each book

## for 0:

def process_zero(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]
    df[['participant_name', 'name']] = df['name'].str.rsplit(' ', 1, expand=True)

    df = df.dropna()
    return df

## for 1:

def process_one(df):
    df = df.copy()
    df.dropna()
    return df[['handicap', 'odds', 'participant_name', 'name']]

## for two and six

def process_two_six(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]

    df[['name', 'participant_name']] = df['name'].str.split(' - ', expand=True)
    df['participant_name'] = df['participant_name'].str.strip()

    df.dropna()

    return df

## for three and five

def process_three_five(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name', 'description']]

    df[['participant_name', 'description', 'a']] = df['description'].str.rsplit(' ', n=2, expand=True)
    df['participant_name'] = df['participant_name'].str.strip()

    df = df.drop(columns=['description', 'a'])
    df = df.dropna()

    return df


## for seven and eight

def process_seven_eight(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]

    df[['participant_name', 'name', 'a']] = df['name'].str.rsplit(' ', n=2, expand=True)

    df = df.drop(columns=['a'])
    df = df.dropna()

    return df

In [3]:
#### for merging all the books

## agg
def agg(df):    
    result_df = df.groupby(['participant_name', 'name']).agg({'handicap': 'first', 'odds': 'median'}).reset_index()
    return result_df

## pivot
def pivot(df):
    df = df.copy()
    pivot_df = pd.pivot_table(df, values='odds', index=['handicap', 'participant_name'], columns='name').reset_index()
    pivot_df.columns = ['line', 'participant_name', 'over_odds', 'under_odds']

    return pivot_df

## calculate odds
## adapted from Ammar Sulmanjee

def calculate_odds(x, y):
    if x >= 0:
        decimal_odds_1 = 1 + x/100
    else: 
        decimal_odds_1 = 1 + 100/abs(x)
        
    if y >= 0:
        decimal_odds_2 = 1 + y/100
    else: 
        decimal_odds_2 = 1 + 100/abs(y)

    imp_prob1 = (1 / decimal_odds_1) * 100
    imp_prob2 = (1 / decimal_odds_2) * 100

    total_implied_prob = round(imp_prob1 + imp_prob2, 4)
    fair_prob1 = round(imp_prob1 / total_implied_prob * 100, 2)
    fair_prob2 = round(imp_prob2 / total_implied_prob * 100, 2)

    ## 47, 53 == 100
    ## 53 - 50 = 3

    return max(fair_prob1, fair_prob2) - 50

In [5]:
## each run takes 10 API calls currently (1 for getting games, 1 for underdog, 8 for sportsbooks)
bets = pd.DataFrame(columns = ['participant_name', 'ev', 'market', 'line'])

## markets we want to check
MARKETS = ['player_assisted_tackles_over_under',
           'player_assists_over_under',
           'player_extra_points_over_under',
           'player_field_goals_over_under',
           'player_interceptions_over_under',
           'player_interceptions_thrown_over_under',
           'player_kicking_points_over_under',
           'player_passing_and_rushing_yds_over_under',
           'player_passing_attempts_over_under',
           'player_passing_completions_over_under',
           'player_passing_tds_over_under',
           'player_passing_yds_over_under',
           'player_receiving_yds_over_under',
           'player_receptions_over_under',
           'player_rushing_and_receiving_yards_over_under',
           'player_rushing_attempts_over_under',
           'player_rushing_yds_over_under',
           'player_sacks_over_under',
           'player_tackles_and_assists_over_under',
           'player_tackles_over_under',
           'player_td_over_under'
           ]

## steals, 

MARKETS = [i for i in MARKETS if i != "player_points_rebounds_over_under"]

## get games
games = get_nfl_games()
game_ids = [i['game_id'] for i in games['games']]

In [12]:
games

{'league': 'nfl',
 'date': '2024-01-28',
 'games': [{'game_id': '5dc3c50e93bc0f8324b16c2323f87e54',
   'away_team': 'Kansas City Chiefs',
   'home_team': 'Baltimore Ravens',
   'start_timestamp': '2024-01-28T20:00:00Z',
   'participants': []},
  {'game_id': 'ac621d714ed4d0d3a71737ed8078c3c3',
   'away_team': 'Detroit Lions',
   'home_team': 'San Francisco 49ers',
   'start_timestamp': '2024-01-28T23:30:00Z',
   'participants': []}]}

In [57]:
def seven_process(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]

    df[['participant_name', 'name', 'a']] = df['name'].str.rsplit(' ', n=2, expand=True)
    df['participant_name'] = df['participant_name'].str.strip()
    df['name'] = df['name'].str.strip()


    df = df.drop(columns=['a'])
    df = df.dropna()

    return df

In [105]:
g = get_nba_games()

In [106]:
g

{'league': 'nba',
 'date': '2024-01-28',
 'games': [{'game_id': '946b81b1664be33cd90243973f5a9a1c',
   'away_team': 'Oklahoma City Thunder',
   'home_team': 'Detroit Pistons',
   'start_timestamp': '2024-01-28T19:00:00Z',
   'participants': []},
  {'game_id': '916dcb706add6762c2f407c994d9e8f0',
   'away_team': 'Memphis Grizzlies',
   'home_team': 'Indiana Pacers',
   'start_timestamp': '2024-01-28T20:30:00Z',
   'participants': []},
  {'game_id': 'b553876d69360a87b409cf9d3038feda',
   'away_team': 'Toronto Raptors',
   'home_team': 'Atlanta Hawks',
   'start_timestamp': '2024-01-28T23:00:00Z',
   'participants': []},
  {'game_id': 'b2c0aba5327ec51369b623f645e839d6',
   'away_team': 'Phoenix Suns',
   'home_team': 'Orlando Magic',
   'start_timestamp': '2024-01-28T23:00:00Z',
   'participants': []},
  {'game_id': 'b8dffc15f667f83d6b336530d33c3749',
   'away_team': 'Chicago Bulls',
   'home_team': 'Portland Trail Blazers',
   'start_timestamp': '2024-01-29T02:00:00Z',
   'participants': 

In [110]:
g = ['946b81b1664be33cd90243973f5a9a1c']
m = ["player_assists_points_over_under"]

In [113]:
book_odds = get_most_recent_odds(g[0], m[0])
books = [book_odds['sportsbooks'][i]['bookie_key'] for i in range(len(book_odds['sportsbooks']))]
books


['fanduel',
 'draftkings',
 'caesars',
 'betmgm',
 'barstool',
 'pointsbet',
 'fliff']

In [97]:
for game in g:
    for market in MARKETS:
        try:
            ud_odds = get_fantasy_lines(game, market)
            book_odds = get_most_recent_odds(game, market)

            ud = pd.DataFrame.from_dict(ud_odds['fantasy_books'][2]['market']['lines'])
            ud = ud[['participant_name', 'line']]
            ud.columns = [ud.columns[0], 'handicap']

            seven = pd.DataFrame.from_dict(book_odds['sportsbooks'][7]['market']['outcomes'])
            seven = seven_process(seven)

            seven = pd.merge(seven, ud, how='inner', on = ['participant_name', 'handicap'])
            seven = pivot(seven)

            seven['ev']= seven.apply(lambda row: calculate_odds(row['over_odds'], row['under_odds']), axis = 1)
            seven = seven.drop(columns=['over_odds', 'under_odds'])
            seven = seven.groupby(['participant_name', 'line']).agg({'ev': 'mean'}).reset_index()

            seven = seven[seven['ev'] > 3]

            if seven.shape[0] > 0:

                names = list(seven.participant_name)
                lines = list(seven.line)
                evs = list(seven.ev)

                for i in range(seven.shape[0]):
                    print(f"FOUND BET: {names[i]} \n market: {market} \n line: {lines[i]} \n ev: {evs[i]}")
        except Exception as e:
            continue

FOUND BET: Lamar Jackson 
 market: player_passing_attempts_over_under 
 line: 29.5 
 ev: 3.0600000000000023
FOUND BET: Patrick Mahomes 
 market: player_passing_tds_over_under 
 line: 1.5 
 ev: 6.329999999999998
FOUND BET: Nelson Agholor 
 market: player_receiving_yds_over_under 
 line: 15.5 
 ev: 3.0600000000000023
FOUND BET: Justice Hill 
 market: player_rushing_attempts_over_under 
 line: 7.5 
 ev: 42.739999999999995
FOUND BET: Lamar Jackson 
 market: player_rushing_yds_over_under 
 line: 65.5 
 ev: 3.0600000000000023


In [88]:
seven = seven[seven['ev'] > 3]

In [89]:
seven.shape

(1, 3)

In [93]:
seven

,participant_name,line,ev
0,Lamar Jackson,65.5,3.06


In [90]:
seven.shape[0] > 0

True

In [10]:
for market in MARKETS:
    for game in game_ids:
        try:
            odds = get_fantasy_lines(game, market)
            print(odds)
            ud = pd.DataFrame.from_dict(odds['fantasy_books'][2]['market']['lines'])
            ud = ud[['participant_name', 'line']]
            ud.columns = [ud.columns[0], 'handicap']

            ## now get all the sportsbooks lines
            odds = get_most_recent_odds(game, market)


            zero = pd.DataFrame.from_dict(odds['sportsbooks'][0]['market']['outcomes'])
            one = pd.DataFrame.from_dict(odds['sportsbooks'][1]['market']['outcomes'])
            two = pd.DataFrame.from_dict(odds['sportsbooks'][2]['market']['outcomes'])
            three = pd.DataFrame.from_dict(odds['sportsbooks'][3]['market']['outcomes'])
            five = pd.DataFrame.from_dict(odds['sportsbooks'][5]['market']['outcomes'])
            six = pd.DataFrame.from_dict(odds['sportsbooks'][6]['market']['outcomes'])
            seven = pd.DataFrame.from_dict(odds['sportsbooks'][7]['market']['outcomes'])
            eight = pd.DataFrame.from_dict(odds['sportsbooks'][8]['market']['outcomes'])

            ## process them
            zero = process_zero(zero)
            one = process_one(one)
            two = process_two_six(two)
            three = process_three_five(three)
            five = process_three_five(five)
            six = process_two_six(six)
            seven = process_seven_eight(seven)
            eight = process_seven_eight(eight)

            ## concat all the books
            datasets = [zero, one, two, three, five, six, seven, eight]
            new_datasets = []

            for dataset in datasets:
                dataset = pd.merge(dataset, ud, how='inner', on = ['participant_name', 'handicap'])
                new_datasets.append(pivot(dataset))

            concat = pd.concat(new_datasets, axis=0)
            final = concat.copy()

            # final = final[final['ev'] >= 3]

            final['market'] = market

            if final.shape[0] > 0: bets = pd.concat([bets, final[['participant_name', 'ev', 'market', 'line']]])
            
        except Exception as e:
            print("exception", e, market)
            continue

Request failed with status: 422
{}
exception 'fantasy_books' player_assisted_tackles_over_under
Request failed with status: 422
{}
exception 'fantasy_books' player_assisted_tackles_over_under
{'game_id': '5dc3c50e93bc0f8324b16c2323f87e54', 'fantasy_books': [{'bookie_key': 'underdog', 'market': {'market_key': 'player_assists_over_under', 'lines': [{'timestamp': '2024-01-28T18:25:24.945726', 'game_id': '5dc3c50e93bc0f8324b16c2323f87e54', 'line': 2.0, 'participant': 11519, 'participant_name': 'Justin Reid'}, {'timestamp': '2024-01-28T18:50:22.434429', 'game_id': '5dc3c50e93bc0f8324b16c2323f87e54', 'line': 3.5, 'participant': 11521, 'participant_name': 'Nick Bolton'}, {'timestamp': '2024-01-28T17:35:20.740162', 'game_id': '5dc3c50e93bc0f8324b16c2323f87e54', 'line': 3.0, 'participant': 11692, 'participant_name': 'Roquan Smith'}, {'timestamp': '2024-01-22T18:40:31.303532', 'game_id': '5dc3c50e93bc0f8324b16c2323f87e54', 'line': 2.5, 'participant': 11693, 'participant_name': 'Patrick Queen'}, 

In [11]:
bets.shape

(0, 4)

In [78]:
bets.to_csv("bets.csv", index=None)

{}